# Was?

**Stand: 17/06/19**

Erstellt ein Dataframe, welcher eine Liste aller zusammenängender Videos enthält. Soll ermöglichen daraus eine Timeline zuerstellen.

In [25]:
%matplotlib inline

import datetime as dt
import json
import os

import iso8601
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import modules.gtimeline_wrapper as gtwrapper

Laden der Video-Filenames und Sortierung.

In [2]:
names = []
with open("./raw_data/00_2016_video_names.txt", "r") as f:
    for line in f:
        names.append(line.strip('\n'))
names.sort()

# Parsen der Daten

In [3]:
df_files = pd.DataFrame(names, columns=['filename'])

In [4]:
def parse_start_time(row):
    cam_str, interval_str = os.path.splitext(row['filename'])[0].split('_')[1:3]

    time_start_str, time_end_str = interval_str.split('--')

    res = iso8601.parse_date(time_start_str)
    return res

def parse_end_time(row):
    cam_str, interval_str = os.path.splitext(row['filename'])[0].split('_')[1:3]

    time_start_str, time_end_str = interval_str.split('--')

    res = iso8601.parse_date(time_end_str)
    #res = int(cam_str)
    return res

def parse_cam_id(row):
    cam_str, interval_str = os.path.splitext(row['filename'])[0].split('_')[1:3]

    time_start_str, time_end_str = interval_str.split('--')

    res = int(cam_str)
    return res

In [5]:
df_files['time_start']= df_files.apply(parse_start_time, axis = 1)
df_files['time_end']= df_files.apply(parse_end_time, axis = 1)
df_files['cam_id']= df_files.apply(parse_cam_id, axis = 1)

## Bestimmen des zeitlichen Abstandes von aufeinaderfolgenden Videos

In [6]:
cam_0 = df_files[df_files['cam_id'] == 0].sort_values('filename')
cam_1 = df_files[df_files['cam_id'] == 1].sort_values('filename')
cam_2 = df_files[df_files['cam_id'] == 2].sort_values('filename')
cam_3 = df_files[df_files['cam_id'] == 3].sort_values('filename')

In [7]:
cam_0['diff_time'] = (cam_0['time_start'] - cam_0.shift(1)['time_end'])
cam_1['diff_time'] = (cam_1['time_start'] - cam_1.shift(1)['time_end'])
cam_2['diff_time'] = (cam_2['time_start'] - cam_2.shift(1)['time_end'])
cam_3['diff_time'] = (cam_3['time_start'] - cam_3.shift(1)['time_end'])

## Bestimmen von 'zusammenhängenden' Videos
Videos, welche einen zeitlichen Abstand geringer `< time_delta` zueinander haben, sollen als zusammenhängend betrachtet werden.

In [8]:
def get_time_intervalls(data_cam, **kwargs):
    df_intervals = []
    curr_time_start = None
    curr_filename_start = None
    curr_time_end = None
    curr_filename_end = None
    count = 0
    for row in data_cam.iterrows():
        row = row[1]
        if pd.isnull(row['diff_time']) and curr_time_start is None and curr_time_end is None:
            curr_time_start = row['time_start']
            curr_filename_start = row['filename']
            curr_time_end = row['time_end']
            curr_filename_end = row['filename']
        elif row['diff_time'] <= dt.timedelta(**kwargs):
            curr_time_end = row['time_end']
            curr_filename_end = row['filename']
        else:
            count += 1
            data = pd.DataFrame([[curr_time_start,curr_time_end, curr_filename_start, curr_filename_end]],
                                columns=['time_start','time_end', 'filename_start', 'filename_end'])
            df_intervals.append(data)
            curr_time_start = row['time_start']
            curr_filename_start = row['filename']
            curr_time_end = row['time_end']
            curr_filename_end = row['filename']
    data = pd.DataFrame([[curr_time_start,curr_time_end, curr_filename_start, curr_filename_end]],
                        columns=['time_start','time_end', 'filename_start', 'filename_end'])
    df_intervals.append(data)
    appended_data = pd.concat(df_intervals, axis=0, ignore_index=True)
    return appended_data

In [9]:
time_delta = {'seconds': 1}

In [10]:
cam_0_intervals = get_time_intervalls(cam_0, **time_delta)
cam_1_intervals = get_time_intervalls(cam_1, **time_delta)
cam_2_intervals = get_time_intervalls(cam_2, **time_delta)
cam_3_intervals = get_time_intervalls(cam_3, **time_delta)

## Vorbereitung für das Darstellen mittels gTimeline

In [11]:
cam_0_intervals['index']=cam_0_intervals.index
cam_1_intervals['index']=cam_1_intervals.index
cam_2_intervals['index']=cam_2_intervals.index
cam_3_intervals['index']=cam_3_intervals.index

cam_0_intervals['cam_id']='Cam_0'
cam_1_intervals['cam_id']='Cam_1'
cam_2_intervals['cam_id']='Cam_2'
cam_3_intervals['cam_id']='Cam_3'

In [12]:
intervals_list = [cam_0_intervals, cam_1_intervals, cam_2_intervals, cam_3_intervals] 
intervals = pd.concat(intervals_list, axis=0, ignore_index=True)

-------------------------------------------

In [13]:
def get_duration(row):
    return row['time_end'] - row['time_start']

In [14]:
intervals['duration']=intervals.apply(get_duration, axis = 1)

In [15]:
def add_tooltip(row):
    string = """
    <table style="font-size:10px">
        <tr>
            <td><b>Index:</b></td><td>{index}</td>
        </tr>
        <tr>
            <td><b>Cam ID:</b></td><td>{cam_id}</td>
        </tr>
        <tr>
            <td><b>Start:</b></td> <td>{time_start}</td>
        </tr>
         <tr>
            <td><b>End:</b></td> <td>{time_end}</td>
        </tr>
        <tr>
            <td><b>Duration:</b></td><td>{duration}</td>
        </tr>
        <tr>
            <td><b>Filename Start:</b></td><td>{filename_start}</td>
        </tr>
        <tr>
            <td><b>Filename End:</b></td><td>{filename_end}</td>
        </tr>
    </table>
    """.format(index=row['index'],
               cam_id=row['cam_id'],
               time_start=row['time_start'],
               time_end=row['time_end'],
               duration=row['duration'],
               filename_start=row['filename_start'],
               filename_end=row['filename_end'])
    return string

In [16]:
intervals['tooltip']=intervals.apply(add_tooltip, axis = 1)

In [17]:
export_intervals = intervals.copy()

In [18]:
export_intervals[:1]

,time_start,time_end,filename_start,filename_end,index,cam_id,duration,tooltip
0,2016-07-19 12:37:31.470185+00:00,2016-07-19 12:38:14.657679+00:00,Cam_0_2016-07-19T12:37:31.470185Z--2016-07-19T...,Cam_0_2016-07-19T12:37:31.470185Z--2016-07-19T...,0,Cam_0,00:00:43.187494,"\n <table style=""font-size:10px"">\n ..."


In [19]:
export_intervals = export_intervals.drop(['filename_start', 'filename_end', 'duration'], axis=1)

In [20]:
export_intervals.columns = ['start', 'end', 'bar_label', 'row_label', 'tooltip']

In [28]:
export_intervals.to_pickle('./derived_data/01_df_cam_intervalls.pkl')

In [23]:
g_dict = gtwrapper.create_gtimeline_dict(export_intervals, 'row_label', 'bar_label', 'start', 'end', 'tooltip')

In [29]:
with open('docs/01_video_timeline/data.json', 'w') as fp:
    json.dump(g_dict, fp)

In [30]:
!firefox -p develop docs/01_video_timeline/01_video_timeline.html